In [6]:
# see documentation https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python
# Prerequisites: 
# - install pocketsphinx 
# - install SpeechRecognition pydub 
# - and possibly install swig and ffmpeg - add to env variables

import speech_recognition as sr
import os
import glob
from pydub import AudioSegment
from pydub.silence import split_on_silence

def get_all_filesname(path):
    files = os.listdir(path)
    return files


# a function that returns the paths to the audio
def get_audio_paths(folder_path):
    filepath_input_list = []

    for filename in glob.iglob("{}".format(folder_path), recursive=True):
        if os.path.isfile(filename): # filter dirs
            #print(filename)
            if not filename.endswith(".txt"):
                filepath_input_list.append(filename)
      
    return filepath_input_list


# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_sphinx(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    
    f = open(os.path.splitext(path)[0] + "{}".format(".txt"),"w+")
    f.write(whole_text)
    f.close()
    # return the text for all chunks detected
    return whole_text

if __name__ == "__main__":
    path = "Y:/TAL/"
    filepath_list = []
    video_filelist = sorted(get_all_filesname(f"{path}/videos"))
    for videonr in video_filelist:
        for filename in get_all_filesname(f"{path}/videos/{videonr}"):
            if filename.endswith(".txt"):
                break
            else:
                filepath_list.append(filename)
            
    wav_list = []
    for element in filepath_list:
        if element.endswith(".wav"):
            wav_list.append(element)
    
    full_path = []
    for element in wav_list:
        full_path.append("Y:\\TAL\\videos\\" + "{}".format(element[0:5]) + "\\" + element)
    
    print(len(full_path))
    
    # create a speech recognition object
    r = sr.Recognizer()
    
    # create the transcripts
    for name in full_path[1:]:
        get_large_audio_transcription(name)
    
    print("Finished")
    

0
Finished
